In [30]:
# Import required libraries

import copy
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [31]:
# Load data
fundamentals = pd.read_csv('../data/raw/fundamentals.csv')
prices = pd.read_csv('../data/raw/prices-split-adjusted.csv')
securities = pd.read_csv('../data/raw/securities.csv')

# Check missing values in the datasets and output that into a csv file
df = pd.DataFrame(fundamentals.isnull().sum())
df.to_csv('outputs/fundamentals_column_error_count.csv')
df = pd.DataFrame(prices.isnull().sum())
df.to_csv('outputs/prices_column_error_count.csv')
df = pd.DataFrame(securities.isnull().sum())
df.to_csv('outputs/securities_column_error_count.csv')


In [32]:
# Drop rows which have missing values in the identified columns

columns_to_check = ['Cash Ratio', 'Current Ratio', 'Quick Ratio', 'For Year', 'Earnings Per Share', 'Estimated Shares Outstanding']
fundamentals = fundamentals.dropna(subset=columns_to_check)

# Drop useless columns
securities.drop(columns=['Date first added'], inplace=True)

In [33]:
# Checking the date formats in the date columns
prices['date'] = pd.to_datetime(prices['date'])
fundamentals['Period Ending'] = pd.to_datetime(fundamentals['Period Ending'])

# Drop duplicates
fundamentals.drop_duplicates(inplace=True)
prices.drop_duplicates(inplace=True)
securities.drop_duplicates(inplace=True)

In [34]:
# Keep rows for ticker symbols which are present in both fundamentals.csv and prices.csv

ticker_symbols_prices = prices['symbol'].unique()
fundamentals = fundamentals[fundamentals['Ticker Symbol'].isin(ticker_symbols_prices)]
ticker_symbols_fundamentals = fundamentals['Ticker Symbol'].unique()
prices = prices[prices['symbol'].isin(ticker_symbols_fundamentals)]
securities = securities[securities['Ticker symbol'].isin(ticker_symbols_fundamentals)]

# Save the final list of ticker symbols
df = pd.DataFrame(ticker_symbols_fundamentals)
df.to_csv('outputs/ticker_symbols.csv')

In [35]:
prices.sort_values(by=['symbol', 'date'], inplace=True)
fundamentals.sort_values(by=['Ticker Symbol', 'Period Ending'], inplace=True)
securities.sort_values(by='Ticker symbol', inplace=True)

In [36]:
# Save the cleaned datasets
fundamentals.to_csv('../data/clean/fundamentals.csv')
prices.to_csv('../data/clean/prices.csv')
securities.to_csv('../data/clean/securities.csv')

In [37]:
# Merge GICS Sector from securities.csv into fundamentals.csv
securities = securities[['Ticker symbol', 'GICS Sector']]
fundamentals = fundamentals.merge(securities, left_on='Ticker Symbol', right_on='Ticker symbol', how='left')
fundamentals = fundamentals.drop(columns=['Ticker symbol'])
fundamentals = fundamentals.rename(columns={'GICS Sector': 'Sector'})

# Create a new column for the next quarter's average stock price
fundamentals['Next Qtr. Avg. Price'] = None
fundamentals['Next Yr. Avg. Price'] = None

# Create a new column for the prev quarter's average stock price
fundamentals['Prev Qtr. Avg. Price'] = None
fundamentals['Prev Yr. Avg. Price'] = None

# Create a new column for the Q avg to Q avg Price % Change
fundamentals['Q avg to Q avg Price % Change'] = None
fundamentals['Y avg to Y avg Price % Change'] = None

# Create a new categorical column for positive or negative price change Q over Q
fundamentals['Up or Down Q over Q'] = None

# Create a new categorical column for positive or negative price change Y over Y
fundamentals['Up or Down Y over Y'] = None

# Create a new categorical column for Buy, Hold, Or Sell, corresponding to 0, 1, and 2 respectively
fundamentals['Buy, Hold, or Sell'] = None

# Calculate the next quarter's average stock price
for index, row in fundamentals.iterrows():
    symbol = row['Ticker Symbol']
    period_ending = row['Period Ending']
    
    # Define the next quarter period
    start_date = period_ending
    end_date = start_date + pd.DateOffset(months=3)

    # Define the next year period
    start_year_date = period_ending
    end_year_date = start_date + pd.DateOffset(months=12)

    # Define the prev quarter period
    prev_end_date = period_ending
    prev_start_date = prev_end_date - pd.DateOffset(months=3)

    # Define the prev year period
    prev_year_end_date = period_ending
    prev_year_start_date = prev_end_date - pd.DateOffset(months=12)
    
    # Get the stock prices for the next quarter
    next_quarter_prices = prices[(prices['symbol'] == symbol) & (prices['date'] > start_date) & (prices['date'] <= end_date)]['close']

    # Get the stock prices for the next year
    next_year_prices = prices[(prices['symbol'] == symbol) & (prices['date'] > start_year_date) & (prices['date'] <= end_year_date)]['close']
    
    # Get the stock prices for the prev quarter
    prev_quarter_prices = prices[(prices['symbol'] == symbol) & (prices['date'] > prev_start_date) & (prices['date'] <= prev_end_date)]['close']

    # Get the stock prices for the prev year
    prev_year_prices = prices[(prices['symbol'] == symbol) & (prices['date'] > prev_year_start_date) & (prices['date'] <= prev_year_end_date)]['close']
    
    # Calculate the rolling average stock price for the next quarter
    if len(next_quarter_prices) > 0: 
        rolling_avg_price = next_quarter_prices.rolling(window=3, min_periods=1).mean().mean()
    else: 
        rolling_avg_price = None
    
    # Calculate the rolling average stock price for the next year
    if len(next_year_prices) > 0: 
        rolling_avg_year_price = next_year_prices.rolling(window=3, min_periods=1).mean().mean()
    else: 
        rolling_avg_year_price = None

    # Calculate the rolling average stock price for the prev quarter
    if len(prev_quarter_prices) > 0: 
        prev_rolling_avg_price = prev_quarter_prices.rolling(window=3, min_periods=1).mean().mean()
    else: 
        prev_rolling_avg_price = None
    
    # Calculate the rolling average stock price for the prev year
    if len(prev_year_prices) > 0: 
        prev_rolling_year_avg_price = prev_year_prices.rolling(window=3, min_periods=1).mean().mean()
    else: 
        prev_rolling_year_avg_price = None
    
    # Update the DataFrame
    fundamentals.at[index, 'Next Qtr. Avg. Price'] = rolling_avg_price
    fundamentals.at[index, 'Next Yr. Avg. Price'] = rolling_avg_year_price
    fundamentals.at[index, 'Prev Qtr. Avg. Price'] = prev_rolling_avg_price
    fundamentals.at[index, 'Prev Yr. Avg. Price'] = prev_rolling_year_avg_price

    if (prev_rolling_avg_price is None) or (rolling_avg_price is None):
        fundamentals.at[index, 'Q avg to Q avg Price % Change'] = None
    else:
        QtoQPercentChange = round(((rolling_avg_price - prev_rolling_avg_price) / prev_rolling_avg_price) * 100, 2)
        fundamentals.at[index, 'Q avg to Q avg Price % Change'] = QtoQPercentChange
        if QtoQPercentChange > 0:
            fundamentals.at[index, 'Up or Down Q over Q'] = 1
        else:
            fundamentals.at[index, 'Up or Down Q over Q'] = 0
    
    if (prev_rolling_year_avg_price is None) or (rolling_avg_year_price is None):
        fundamentals.at[index, 'Y avg to Y avg Price % Change'] = None
    else:
        YtoYPercentChange = round(((rolling_avg_year_price - prev_rolling_year_avg_price) / prev_rolling_year_avg_price) * 100, 2)
        fundamentals.at[index, 'Y avg to Y avg Price % Change'] = YtoYPercentChange
        if YtoYPercentChange > 0:
            fundamentals.at[index, 'Up or Down Y over Y'] = 1
        else:
            fundamentals.at[index, 'Up or Down Y over Y'] = 0
        if YtoYPercentChange >= 8:
            fundamentals.at[index, 'Buy, Hold, or Sell'] = 0
        elif -8 < YtoYPercentChange < 8:
            fundamentals.at[index, 'Buy, Hold, or Sell'] = 1
        elif YtoYPercentChange <= -8:
            fundamentals.at[index, 'Buy, Hold, or Sell'] = 2

# Convert columns to numeric, except 'Ticker Symbol', 'Period Ending', and 'Sector'
cols_to_convert = [col for col in fundamentals.columns if col not in ['Ticker Symbol', 'Period Ending', 'Sector', 'Up or Down Q over Q', 'Up or Down Y over Y', 'Buy, Hold, or Sell']]
fundamentals[cols_to_convert] = fundamentals[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Drop redundant first column
fundamentals = fundamentals.drop([fundamentals.columns[0], 'Next Qtr. Avg. Price', 'Next Yr. Avg. Price', 'Prev Qtr. Avg. Price', 'Prev Yr. Avg. Price'], axis=1)


In [38]:
scale = StandardScaler()
columnsNotToBeScaled = ['Ticker Symbol','Period Ending', 'Sector', 'Q avg to Q avg Price % Change', 'Y avg to Y avg Price % Change', 'Up or Down Q over Q', 'Up or Down Y over Y', 'Buy, Hold, or Sell']

fundamentals_scaled = copy.deepcopy(fundamentals)

for column in fundamentals_scaled.columns:
    if column not in columnsNotToBeScaled:
        fundamentals_scaled[column] = scale.fit_transform(pd.DataFrame(fundamentals_scaled[column]))


In [39]:
# Save the final merged datasets
fundamentals.to_csv('../data/merged_data.csv', index=False)
fundamentals_scaled.to_csv('../data/merged_scaled_data.csv', index=False)